# Bio Weavers MVP — Demo Notebook

Quick walkthrough of the proof-of-life pipeline.

In [2]:
import sys
from pathlib import Path

# Ensure repo root is on sys.path
repo_root = Path.cwd().resolve()
if (repo_root / 'src').exists():
    sys.path.insert(0, str(repo_root))
elif (repo_root.parent / 'src').exists():
    sys.path.insert(0, str(repo_root.parent))


In [3]:
from pathlib import Path

from src.geometry import load_boundary, create_buffer, get_bounding_box
from src.species import query_inaturalist, get_unique_species, filter_federally_listed
from src.export import export_species_csv, export_species_excel


In [4]:
# Load sample boundary
boundary_path = Path("../data/sample_boundary.geojson")
boundary = load_boundary(boundary_path)
boundary.head()

,name,geometry
0,Sample Project Area,"POLYGON ((-119.85 34.42, -119.82 34.42, -119.8..."


In [5]:
# Create a 5-mile buffer and bounding box
buffered = create_buffer(boundary, distance_miles=5)
bbox = get_bounding_box(buffered)
bbox

(np.float64(-119.93759914482474),
 np.float64(34.34748591293555),
 np.float64(-119.73240085517526),
 np.float64(34.5225067116467))

In [6]:
# Query iNaturalist (network call)
observations = query_inaturalist(bbox, limit=50)
observations.head()

,observation_id,observed_on,scientific_name,common_name,taxon_id,iconic_taxon,conservation_status,latitude,longitude,place_guess
0,338005382,2026-02-09,Bucephala albeola,Bufflehead,6993,Aves,None,34.440362,-119.848358,"Lake Los Carneros, Goleta, CA, US"
1,337977238,2026-02-09,Hyles lineata,White-lined Sphinx,49348,Insecta,None,34.413329,-119.860972,"Abrego & Camino Pescadero, Isla Vista, CA 9311..."
2,337921706,2026-02-08,Nannopterum auritum,Double-crested Cormorant,1454382,Aves,None,34.411820,-119.876908,"Goleta, CA, US"
3,337921734,2026-02-08,Ardea herodias,Great Blue Heron,4956,Aves,None,34.411833,-119.876883,"Goleta, CA, US"
4,337887830,2026-02-08,Ribes amarum,Bitter Gooseberry,53433,Plantae,None,34.424322,-119.750025,"Hidden Valley Park, Santa Barbara, CA, US"


In [7]:
# Build species list + filter federally listed
species = get_unique_species(observations)
listed = filter_federally_listed(species)
species.head()

,scientific_name,common_name,conservation_status,observation_count,first_observed,last_observed
0,Anarhynchus nivosus,Snowy Plover,nt,5,2026-02-08,2026-02-08
1,Danaus plexippus,Monarch,g4,2,2026-02-07,2026-02-08
2,Larus heermanni,Heermann's Gull,nt,1,2026-02-08,2026-02-08


In [8]:
# Export outputs
output_dir = Path("../output")
export_species_csv(species, output_dir / "species.csv", project_name="Sample Project")
export_species_excel(species, listed, output_dir / "species.xlsx", project_name="Sample Project")


PosixPath('../output/species.xlsx')